# Ether, Token(ERC20) Transfers

### Dataset Information
1. tx_date, tx_count, tx_volume_Ether, tx_count_ERC20, tx_tx_volume_ERC20
> data from Google Bigquery
>
> https://cloud.google.com/blog/products/data-analytics/ethereum-bigquery-public-dataset-smart-contract-analytics

    1-1.'Transactions' data parameters
    
        from_address : String, address of the sender
    
        to_address : String, address of the receiver. null when its a contract creation transaction
    
        value : Numeric, value transferred in Wei

    1-2. 'token_transfers' data parameters
    
        from_address : String, address of the sender
    
        to_address : String, address of the receiver.
    
        value : String, amount of tokens transferred(ERC20) / id of the token transferred(ERC721)



2. MarketCap, Price, Volume in dollar term
> CoinMarketCap
>
> https://coinmarketcap.com/currencies/ethereum/historical-data/?start=20160101&end=20181231
>
> 참고 : API 사용은 latest 정보만 무료로 볼 수 있음, historical data는 우회적으로 크롤링함
>
> Volume, Price제외 각 $당 가격은 OHLCV 중 Close value로 계산

#주의! # 맨 첫 시작 코드 (다시 돌리면 에러 발생)

from google.cloud import storage

# Instantiates a client
storage_client = storage.Client()

# The name for the new bucket
bucket_name = 'ethereumjulie'

# Creates the new bucket
bucket = storage_client.create_bucket(bucket_name)

print('Bucket {} created.'.format(bucket.name))

In [1]:
from google.cloud import bigquery
import pandas as pd

client = bigquery.Client()

In [3]:
sql = """
SELECT 
  DATE_TRUNC(DATE(transactions.block_timestamp), DAY) AS tx_date, 
  COUNT(*) AS tx_count, 
  SUM(transactions.value/POWER(10,18)) AS Tx_volume_Ether
FROM `bigquery-public-data.ethereum_blockchain.transactions` AS transactions
WHERE DATE(transactions.block_timestamp) >= DATE('2015-10-30') and DATE(transactions.block_timestamp) <= DATE('2019-04-29')
GROUP BY tx_date
ORDER by tx_date ASC
"""
df = client.query(sql).to_dataframe()
df.head()

,tx_date,tx_count,Tx_volume_Ether
0,2015-10-30,7941,1.209372e+06
1,2015-10-31,7557,2.764179e+05
2,2015-11-01,6915,1.455244e+05
3,2015-11-02,6558,3.518903e+05
4,2015-11-03,7399,5.951720e+05


In [4]:
sql = """
SELECT 
  DATE_TRUNC(DATE(transactions.block_timestamp), DAY) AS tx_date, 
  COUNT(*) AS tx_count_ERC20, 
  SUM(CAST(transactions.value AS FLOAT64)/POWER(10,18)) AS Tx_volume_ERC20
FROM `bigquery-public-data.ethereum_blockchain.token_transfers` AS transactions
WHERE DATE(transactions.block_timestamp) >= DATE('2015-10-30') and DATE(transactions.block_timestamp) <= DATE('2019-04-29')
GROUP BY tx_date
ORDER by tx_date ASC
"""
df_ERC20 = client.query(sql).to_dataframe()
df_ERC20.head()

,tx_date,tx_count_ERC20,Tx_volume_ERC20
0,2015-10-30,3,2.000000e-15
1,2015-11-02,1,5.000000e-16
2,2015-11-03,30,8.001248e-07
3,2015-11-04,18,3.006000e-04
4,2015-11-05,12,5.102010e-04


In [5]:
from bs4 import BeautifulSoup
import requests

url="https://coinmarketcap.com/currencies/ethereum/historical-data/?start=20151030&end=20190429"
content = requests.get(url).content
soup = BeautifulSoup(content,'html.parser')
table = soup.find('table', {'class': 'table'})

In [6]:
data = [[td.text.strip() for td in tr.findChildren('td')] 
        for tr in table.findChildren('tr')]

In [7]:
df_price = pd.DataFrame(data)
df_price.drop(df_price.index[0], inplace=True) # first row is empty
df_price[0] =  pd.to_datetime(df_price[0]) # date
for i in range(1,7):
    df_price[i] = pd.to_numeric(df_price[i].str.replace(",","").str.replace("-","")) # some vol is missing and has -
df_price.columns = ['Date','Open','High','Low','Close','Volume','Market Cap']
df_price.set_index('Date',inplace=True)
df_price.sort_index(inplace=True)

In [8]:
df_price.head()

,Open,High,Low,Close,Volume,Market Cap
Date,,,,,,
2015-10-30,1.210000,1.35,0.968160,1.040000,2429200,77401817
2015-10-31,1.050000,1.06,0.900171,0.916627,673892,68163368
2015-11-01,0.920847,1.08,0.899082,1.060000,588913,78530263
2015-11-02,1.060000,1.10,0.953647,0.989789,1145200,73654327
2015-11-03,0.994568,1.07,0.936615,1.010000,1907690,75434114


In [9]:
df_price_2 = df_price.copy()

In [10]:
df_price = df_price.drop(['Open', 'High', 'Low', 'Close'], axis=1)

In [11]:
result = pd.merge(df, df_ERC20, how='outer', on="tx_date")
result = result.fillna(0)

In [12]:
df_price.reset_index(level=0, inplace=True)

In [13]:
df_price.columns = ['tx_date', 'Volume($)', 'Market Cap($)']

In [14]:
result['tx_date'] = pd.to_datetime(result['tx_date'])

In [15]:
result = pd.merge(result, df_price, how='inner')
result = result.fillna(0)

In [16]:
easy = result.copy()

In [17]:
# 보기 편하게 천단위마다 , 포맷팅
easy['Volume($)'] = easy['Volume($)'].apply("{:,}".format)
easy['Market Cap($)'] = easy['Market Cap($)'].apply("{:,}".format)
easy

,tx_date,tx_count,Tx_volume_Ether,tx_count_ERC20,Tx_volume_ERC20,Volume($),Market Cap($)
0,2015-10-30,7941,1.209372e+06,3.0,2.000000e-15,"2,429,200","77,401,817"
1,2015-10-31,7557,2.764179e+05,0.0,0.000000e+00,"673,892","68,163,368"
2,2015-11-01,6915,1.455244e+05,0.0,0.000000e+00,"588,913","78,530,263"
3,2015-11-02,6558,3.518903e+05,1.0,5.000000e-16,"1,145,200","73,654,327"
4,2015-11-03,7399,5.951720e+05,30.0,8.001248e-07,"1,907,690","75,434,114"
5,2015-11-04,7037,8.052793e+05,18.0,3.006000e-04,"3,060,340","66,947,930"
6,2015-11-05,7498,3.441772e+05,12.0,5.102010e-04,"1,179,840","66,716,645"
7,2015-11-06,7882,9.381777e+05,0.0,0.000000e+00,"913,666","69,004,595"
8,2015-11-07,10996,4.259684e+05,0.0,0.000000e+00,"890,187","69,172,171"
9,2015-11-08,12413,8.966892e+05,18.0,2.000009e-10,"1,021,460","76,614,488"


# DAU, MAU, CUM

In [19]:
sql = """
SELECT 
  DATE_TRUNC(DATE(transactions.block_timestamp), DAY) AS tx_date,
  COUNT(transactions.from_address) AS DAU
  
FROM `bigquery-public-data.ethereum_blockchain.transactions` AS transactions
WHERE 
  DATE(transactions.block_timestamp) >= DATE('2015-10-30') and DATE(transactions.block_timestamp) <= DATE('2019-04-29')
  AND transactions.to_address is not null
  AND transactions.value > 0
  
GROUP BY tx_date
ORDER by tx_date ASC
"""
df_DAU = client.query(sql).to_dataframe()
df_DAU.head()

,tx_date,DAU
0,2015-10-30,7501
1,2015-10-31,7076
2,2015-11-01,6516
3,2015-11-02,6223
4,2015-11-03,6973


In [20]:
df_DAU['tx_date'] = pd.to_datetime(df_DAU['tx_date'])

In [27]:
result = pd.merge(result, df_DAU, how='outer', on="tx_date")

In [30]:
result = result.drop("DAU_y", axis=1)

In [31]:
result.head()

,tx_date,tx_count,Tx_volume_Ether,tx_count_ERC20,Tx_volume_ERC20,Volume($),Market Cap($),DAU_x,CUM
0,2015-10-30,7941,1.209372e+06,3.0,2.000000e-15,2429200,77401817,7501,7501
1,2015-10-31,7557,2.764179e+05,0.0,0.000000e+00,673892,68163368,7076,14577
2,2015-11-01,6915,1.455244e+05,0.0,0.000000e+00,588913,78530263,6516,21093
3,2015-11-02,6558,3.518903e+05,1.0,5.000000e-16,1145200,73654327,6223,27316
4,2015-11-03,7399,5.951720e+05,30.0,8.001248e-07,1907690,75434114,6973,34289


In [34]:
result = result.rename({"DAU_x" : "DAU"}, axis=1)

In [35]:
result.to_csv("result.csv")

In [21]:
pd.DatetimeIndex(df_DAU.tx_date).to_period("M")
per = df_DAU.tx_date.dt.to_period("M")
g = df_DAU.groupby(per)
df_MAU = pd.DataFrame(g.sum())
df_MAU.columns = ['MAU']
df_MAU.reset_index(level=0, inplace=True)

In [22]:
df_MAU.head()

,tx_date,MAU
0,2015-10,14577
1,2015-11,210195
2,2015-12,282711
3,2016-01,344809
4,2016-02,478424


In [23]:
df_CUM = df_DAU.copy()
df_CUM['DAU'] = df_CUM['DAU'].cumsum()
df_CUM.columns = ['tx_date', 'CUM']
df_CUM['tx_date'] = pd.to_datetime(df_CUM['tx_date'])

In [24]:
df_CUM.head()

,tx_date,CUM
0,2015-10-30,7501
1,2015-10-31,14577
2,2015-11-01,21093
3,2015-11-02,27316
4,2015-11-03,34289


# Final Result

In [25]:
result = pd.merge(result, df_DAU, how='inner')
result = pd.merge(result, df_CUM, how='inner')

In [26]:
df_price_2.reset_index(level=0, inplace=True)

In [25]:
result['Price'] = df_price_2.Close

In [26]:
result['Tx_volume($)'] = result['Tx_volume_Ether'] * result['Price']
result['Tx_volume_ERC20($)'] = result['Tx_volume_ERC20'] * result['Price']

In [27]:
result.drop('Price', axis=1, inplace=True)

In [28]:
result = result[['tx_date', 'tx_count', 'Tx_volume_Ether', 'Tx_volume($)', 'tx_count_ERC20', 'Tx_volume_ERC20', 'Tx_volume_ERC20($)', 'Volume($)', 'Market Cap($)', 'DAU', 'CUM']]

In [29]:
result.to_csv("result.csv")

# Linear Regression

X : 
tx_count, Tx_volume_Ether, tx_count_ERC20, Tx_volume_ERC20, Volume(), DAU, CUM, MAU, Tx_volume(), Tx_volume_ERC20()

Y :
Market Cap($)

In [30]:
result.head()

,tx_date,tx_count,Tx_volume_Ether,Tx_volume($),tx_count_ERC20,Tx_volume_ERC20,Tx_volume_ERC20($),Volume($),Market Cap($),DAU,CUM
0,2016-01-01,8233,3.445478e+05,326639.542431,0.0,0.0,0.0,206062,71980386,8006,8006
1,2016-01-02,9164,2.592400e+05,242940.035122,0.0,0.0,0.0,255504,71176658,8814,16820
2,2016-01-03,9258,3.708200e+05,360401.805781,0.0,0.0,0.0,407632,73843292,9019,25839
3,2016-01-04,9474,1.541255e+05,147109.680450,0.0,0.0,0.0,346245,72543707,9059,34898
4,2016-01-05,16430,1.036291e+06,984659.012911,0.0,0.0,0.0,219833,72240974,11164,46062


In [31]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [32]:
x_lst = result.columns.tolist()
x_lst.remove('tx_date')
x_lst.remove('Market Cap($)')

In [33]:
def make_result(x_lst, target):
    coef_lst = []
    score_lst = []
    for x in x_lst:
        X = result[x].values
        y = result[target].values
        X = X.reshape(len(X), 1)
        y = y.reshape(len(y), 1)
        regr = linear_model.LinearRegression()
        regr.fit(X, y)
        coef_lst.append(regr.coef_[0][0])
        score_lst.append(regr.score(X, y))
    return coef_lst, score_lst

In [34]:
coef_lst, score_lst = make_result(x_lst, 'Market Cap($)')

In [35]:
empty_lst = [0] * len(x_lst)
final = pd.DataFrame({'Coef' : empty_lst, 'Score' : empty_lst})
final['x_lst'] = x_lst
final.Coef = coef_lst
final.Score = score_lst
final = final.set_index('x_lst')
final.to_csv("final.csv")